In [204]:
%matplotlib widget
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import gsw

In [205]:
file_list = glob(ctd_dir+'*CtdDer.txt')

#ctd_dir = 'data/CTD_data_Station_M/'
#filename = '2006347tibrCtdDer.txt'

In [ ]:
mlat = 35+8.4585/60  
mlon = -122-59.9036/60

In [206]:
dz = 1
salspike_thresh = 0.2

In [207]:
plt.figure()
plt.gca().invert_yaxis()

dflist = []

for i,filename in enumerate(file_list):

    try:
        df = pd.read_csv(filename)
        
        
        #spi, = np.where((np.abs(np.diff(df['salin']))>0.2) | (np.abs(np.diff(df['pres']))>10))
        spi, = np.where((np.abs(np.diff(df['salin'][:-1]))>salspike_thresh) & 
                        (np.abs(np.diff(df['salin'][1:]))>salspike_thresh))
        spi = spi+1
        
        df['salin'][spi] = np.nan
        df['sigmat'][spi] = np.nan
        df['sigmatheta'][spi] = np.nan
        
        mSA_am = gsw.SA_from_SP(df['salin'], df['pres'], mlon, mlat)
        mCT_am = gsw.CT_from_t(mSA_am,mtemp_an,mpr)
        msigma0 = gsw.sigma0(mSA_am,mCT_am)
        msigma4 = gsw.sigma4(mSA_am,mCT_am)

mNsq,mpr_mid = gsw.Nsquared(mSA_am,mCT_am,mpr,lat=temp['lat'][ii])
mN = np.sqrt(mNsq)
        
        
        df['zp'] = np.floor(df['pres']/dz+0.5)*dz
        dfm = df.groupby('zp').mean()
        
        dfm['sigmatheta'] = dfm['sigmatheta'].rolling(10).median()
        dfm['sigmatheta'] = dfm['sigmatheta'].rolling(25,win_type='hanning').mean()
        
        if i==0:
            dfall = dfm
        else:
            dfall = dfall.append(dfm)
        
        plt.plot(dfm['sigmatheta'],dfm.index)
    except:
        pass

FigureCanvasNbAgg()

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [191]:
dfmean = dfm.groupby('zp').mean()

plt.figure()
plt.plot(dfall['sigmatheta'],dfall.index,linewidth=1,color='gray')
plt.plot(dfmean['sigmatheta'],dfmean.index,linewidth=2,color='k')
plt.gca().invert_yaxis()

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [202]:
f = gsw.f(35+8.4585/60)

sigmatheta = np.array(dfmean['sigmatheta'])
zp = np.array(dfmean.index)
drhodz = -(sigmatheta[2:]-sigmatheta[:-2])/(zp[2:]-zp[:-2])
z = 0.5*(zp[2:]+zp[:-2])

N = np.sqrt(-9.8*drhodz/1025)

In [201]:
plt.figure()
plt.semilogx(-drhodz[:-1],z[:-1])
yl = plt.ylim()
plt.semilogx([f,f],yl,'k--')
plt.ylim(yl)
plt.gca().invert_yaxis()

FigureCanvasNbAgg()

In [203]:
plt.figure()
plt.plot(-drhodz[:-1],z[:-1])
yl = plt.ylim()
plt.plot([f,f],yl,'k--')
plt.ylim(yl)
plt.gca().invert_yaxis()

FigureCanvasNbAgg()

In [154]:
len(np.array(dfmean.index))

1201

In [68]:
plt.figure()
plt.hist(np.diff(df['salin']),100)

FigureCanvasNbAgg()

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/numpy/lib/histograms.py:754: RuntimeWarning: invalid value encountered in greater_equal
  keep = (tmp_a >= first_edge)
/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/numpy/lib/histograms.py:755: RuntimeWarning: invalid value encountered in less_equal
  keep &= (tmp_a <= last_edge)


(array([1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 2.000e+00,
        0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 2.000e+00,
        0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00,
        1.000e+00, 0.000e+00, 1.000e+00, 0.000e+00, 1.000e+00, 1.000e+00,
        1.000e+00, 1.000e+00, 2.000e+00, 3.000e+00, 2.000e+00, 1.000e+00,
        2.000e+00, 1.000e+00, 3.000e+00, 2.000e+00, 2.000e+00, 5.000e+00,
        2.000e+00, 1.100e+01, 5.000e+00, 1.800e+01, 1.100e+01, 2.200e+01,
        3.500e+01, 5.700e+01, 4.770e+02, 1.102e+03, 1.210e+02, 3.500e+01,
        2.100e+01, 1.300e+01, 2.100e+01, 1.100e+01, 7.000e+00, 3.000e+00,
        5.000e+00, 5.000e+00, 6.000e+00, 3.000e+00, 3.000e+00, 2.000e+00,
        1.000e+00, 0.000e+00, 1.000e+00, 0.000e+00, 2.000e+00, 2.000e+00,
        2.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 0.000e+00,
        1.000e+00, 0.000e+00, 0.000e+00, 2.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+0

0          NaN
1          NaN
2          NaN
3          NaN
4       34.484
5       34.484
6       34.483
7       34.483
8       34.483
9       34.483
10      34.483
11      34.483
12      34.483
13      34.483
14      34.483
15      34.483
16      34.483
17      34.483
18      34.483
19      34.483
20      34.482
21      34.482
22      34.482
23      34.482
24      34.482
25      34.481
26      34.481
27      34.481
28      34.481
29      34.481
         ...  
2043    34.455
2044    34.455
2045    34.454
2046    34.454
2047    34.454
2048    34.453
2049    34.453
2050    34.453
2051    34.453
2052    34.452
2053    34.452
2054    34.451
2055    34.451
2056    34.451
2057    34.450
2058    34.450
2059    34.450
2060    34.449
2061    34.448
2062    34.448
2063    34.447
2064    34.447
2065    34.446
2066    34.446
2067    34.445
2068    34.445
2069    34.444
2070    34.444
2071    34.443
2072    34.443
Name: salin, Length: 2073, dtype: float64

In [184]:
dfm

,usec,year,day,pres,temp,cond,o2curr,o2temp,o2delta,transvolt,...,lon,depth,salin,theta,oxyg,light,beac,sound,sigmat,sigmatheta
zp,,,,,,,,,,,,,,,,,,,,,
4.0,1.190310e+09,2007.0,263.0,3.900000,16.436000,4.203640,2.729500,0.0,NaN,4.109600,...,-123.125327,3.800000,32.977000,16.435000,5.540000,83.300000,0.729000,1508.900000,24.094000,NaN
9.0,1.190310e+09,2007.0,263.0,8.600000,16.458000,4.209020,2.730900,0.0,NaN,4.191000,...,-123.125327,8.600000,33.005000,16.456000,5.540000,85.000000,0.650000,1509.000000,24.110000,NaN
11.0,1.190254e+09,2007.0,263.0,10.700000,16.469000,4.208810,2.753300,0.0,NaN,4.179700,...,-123.015653,10.700000,32.992000,16.468000,5.600000,84.800000,0.660000,1509.100000,24.098000,NaN
13.0,1.190262e+09,2007.0,263.0,13.028571,16.452571,4.209101,2.748800,0.0,NaN,4.177586,...,-123.027163,12.928571,33.008000,16.450429,5.588571,84.728571,0.662571,1509.071429,24.113714,NaN
15.0,1.190310e+09,2007.0,263.0,14.700000,16.415000,4.224450,2.736600,0.0,NaN,4.184500,...,-123.016010,14.600000,33.173000,16.413000,5.560000,84.900000,0.656000,1509.200000,24.249000,NaN
16.0,1.190254e+09,2007.0,263.0,15.900000,16.198000,4.201980,2.740900,0.0,NaN,4.180000,...,-123.015653,15.800000,33.155000,16.196000,5.590000,84.800000,0.660000,1508.500000,24.285000,NaN
19.0,1.190273e+09,2007.0,263.0,19.066667,16.016000,4.195837,2.735233,0.0,NaN,4.183733,...,-123.015771,18.900000,33.251667,16.013333,5.593333,84.833333,0.656667,1508.133333,24.400000,NaN
20.0,1.190282e+09,2007.0,263.0,19.650000,16.022000,4.204735,2.742500,0.0,NaN,4.187050,...,-123.015842,19.550000,33.325500,16.019000,5.610000,84.900000,0.653000,1508.200000,24.455500,NaN
21.0,1.190310e+09,2007.0,263.0,21.000000,16.166000,4.223560,2.752650,0.0,NaN,4.183350,...,-123.015962,20.850000,33.371000,16.163000,5.620000,84.850000,0.657000,1508.800000,24.458000,NaN
